In [1]:
# Adjust Python path within the notebook
import sys
project_root = '/Users/thangnguyen/Documents/GitHub/project-1-individual-knam2609'
if project_root not in sys.path:
    sys.path.insert(0, project_root)

import scripts

In [2]:
# Create SparkSession
spark = scripts.clean_base.create_spark_session()

24/08/27 21:50:56 WARN Utils: Your hostname, THANGs-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 100.86.89.69 instead (on interface en0)
24/08/27 21:50:56 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/27 21:50:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# Create directories for plots
scripts.download.make_directories(["../plots/yellow/"], ["map"])

In [4]:
import pandas as pd
import geopandas as gpd
import folium

In [5]:
# sf stands for shape file
sf = gpd.read_file("../data/landing/taxi_zones/taxi_zones.shp")
zones = pd.read_csv("../data/landing/taxi_zones.csv")

sf.head()

,OBJECTID,Shape_Leng,Shape_Area,zone,LocationID,borough,geometry
0,1,0.116357,0.000782,Newark Airport,1,EWR,"POLYGON ((933100.918 192536.086, 933091.011 19..."
1,2,0.433470,0.004866,Jamaica Bay,2,Queens,"MULTIPOLYGON (((1033269.244 172126.008, 103343..."
2,3,0.084341,0.000314,Allerton/Pelham Gardens,3,Bronx,"POLYGON ((1026308.77 256767.698, 1026495.593 2..."
3,4,0.043567,0.000112,Alphabet City,4,Manhattan,"POLYGON ((992073.467 203714.076, 992068.667 20..."
4,5,0.092146,0.000498,Arden Heights,5,Staten Island,"POLYGON ((935843.31 144283.336, 936046.565 144..."


In [6]:
# Convert the geometry shape to latitude and longitude
sf['geometry'] = sf['geometry'].to_crs("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
sf.head()

,OBJECTID,Shape_Leng,Shape_Area,zone,LocationID,borough,geometry
0,1,0.116357,0.000782,Newark Airport,1,EWR,"POLYGON ((-74.18445 40.695, -74.18449 40.6951,..."
1,2,0.433470,0.004866,Jamaica Bay,2,Queens,"MULTIPOLYGON (((-73.82338 40.63899, -73.82277 ..."
2,3,0.084341,0.000314,Allerton/Pelham Gardens,3,Bronx,"POLYGON ((-73.84793 40.87134, -73.84725 40.870..."
3,4,0.043567,0.000112,Alphabet City,4,Manhattan,"POLYGON ((-73.97177 40.72582, -73.97179 40.725..."
4,5,0.092146,0.000498,Arden Heights,5,Staten Island,"POLYGON ((-74.17422 40.56257, -74.17349 40.562..."


In [7]:
zones.head()

,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone


In [8]:
gdf = gpd.GeoDataFrame(
    pd.merge(zones, sf, on='LocationID', how='inner')
)

gdf.head()

,LocationID,Borough,Zone,service_zone,OBJECTID,Shape_Leng,Shape_Area,zone,borough,geometry
0,1,EWR,Newark Airport,EWR,1,0.116357,0.000782,Newark Airport,EWR,"POLYGON ((-74.18445 40.695, -74.18449 40.6951,..."
1,2,Queens,Jamaica Bay,Boro Zone,2,0.433470,0.004866,Jamaica Bay,Queens,"MULTIPOLYGON (((-73.82338 40.63899, -73.82277 ..."
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone,3,0.084341,0.000314,Allerton/Pelham Gardens,Bronx,"POLYGON ((-73.84793 40.87134, -73.84725 40.870..."
3,4,Manhattan,Alphabet City,Yellow Zone,4,0.043567,0.000112,Alphabet City,Manhattan,"POLYGON ((-73.97177 40.72582, -73.97179 40.725..."
4,5,Staten Island,Arden Heights,Boro Zone,5,0.092146,0.000498,Arden Heights,Staten Island,"POLYGON ((-74.17422 40.56257, -74.17349 40.562..."


In [9]:
# create a JSON 
geoJSON = gdf[['LocationID', 'geometry']].drop_duplicates('LocationID').to_json()

# print the first 300 chars of the json
print(geoJSON[:300])

{"type": "FeatureCollection", "features": [{"id": "0", "type": "Feature", "properties": {"LocationID": 1}, "geometry": {"type": "Polygon", "coordinates": [[[-74.18445299999996, 40.694996000000096], [-74.18448899999999, 40.695095000000066], [-74.18449799999996, 40.69518500000006], [-74.18438099999997


In [10]:
yellow_df = pd.read_parquet("../data/curated/yellow/yellow.parquet")

In [11]:
yellow_df

,VendorID,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,trip_time,fare_per_miles
0,2,2023-08-12 13:04:05,2023-08-12 13:09:43,1,0.91,1,141,141,1,7.2,0.0,0.5,1.50,0.0,1.0,12.70,2.5,0.00,5.633333,13.956044
1,2,2023-08-12 13:22:12,2023-08-12 13:27:23,1,1.09,1,142,43,1,7.9,0.0,0.5,2.38,0.0,1.0,14.28,2.5,0.00,5.183333,13.100917
2,1,2023-08-12 13:14:57,2023-08-12 13:33:55,1,3.40,1,236,107,1,18.4,2.5,0.5,4.50,0.0,1.0,26.90,2.5,0.00,18.966667,7.911765
3,2,2023-08-12 13:17:08,2023-08-12 14:05:16,1,16.91,2,132,161,1,70.0,0.0,0.5,8.00,0.0,1.0,83.75,2.5,1.75,48.133333,4.952691
4,2,2023-08-12 13:21:00,2023-08-12 13:30:28,1,1.65,1,43,161,1,11.4,0.0,0.5,3.08,0.0,1.0,18.48,2.5,0.00,9.466667,11.200000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1707999,2,2023-07-31 23:33:48,2023-07-31 23:43:37,1,1.10,1,90,68,2,10.7,1.0,0.5,0.00,0.0,1.0,15.70,2.5,0.00,9.816667,14.272727
1708000,1,2023-07-31 23:25:44,2023-07-31 23:47:56,1,6.00,1,163,42,2,27.5,3.5,0.5,0.00,0.0,1.0,32.50,2.5,0.00,22.200000,5.416667
1708001,2,2023-07-31 23:52:33,2023-07-31 23:56:09,1,0.43,1,162,233,1,5.1,1.0,0.5,2.02,0.0,1.0,12.12,2.5,0.00,3.600000,28.186047
1708002,2,2023-07-31 23:27:45,2023-07-31 23:35:21,1,1.09,1,125,79,1,9.3,1.0,0.5,2.86,0.0,1.0,17.16,2.5,0.00,7.600000,15.743119


In [12]:
yellow_df = yellow_df \
    .merge(gdf[['LocationID', 'geometry']], left_on='PULocationID', right_on='LocationID') \
    .drop('LocationID', axis=1)

yellow_df.head()


,VendorID,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,...,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,trip_time,fare_per_miles,geometry
0,2,2023-08-12 13:04:05,2023-08-12 13:09:43,1,0.91,1,141,141,1,7.2,...,0.5,1.50,0.0,1.0,12.70,2.5,0.0,5.633333,13.956044,"POLYGON ((-73.96178 40.75988, -73.96197 40.759..."
1,1,2023-08-12 13:29:08,2023-08-12 13:31:22,1,0.70,1,141,141,1,5.8,...,0.5,1.95,0.0,1.0,11.75,2.5,0.0,2.233333,16.785714,"POLYGON ((-73.96178 40.75988, -73.96197 40.759..."
2,1,2023-08-12 14:18:00,2023-08-12 14:25:39,3,1.00,1,141,237,1,9.3,...,0.5,2.65,0.0,1.0,15.95,2.5,0.0,7.650000,15.950000,"POLYGON ((-73.96178 40.75988, -73.96197 40.759..."
3,2,2023-08-12 14:06:41,2023-08-12 14:07:54,1,0.35,1,141,262,1,3.7,...,0.5,1.54,0.0,1.0,9.24,2.5,0.0,1.216667,26.400000,"POLYGON ((-73.96178 40.75988, -73.96197 40.759..."
4,2,2023-08-12 14:40:08,2023-08-12 14:42:45,1,0.41,1,141,263,2,5.1,...,0.5,0.00,0.0,1.0,9.10,2.5,0.0,2.616667,22.195122,"POLYGON ((-73.96178 40.75988, -73.96197 40.759..."


In [13]:
yellow_df.count()

VendorID                 1694028
pickup_datetime          1694028
dropoff_datetime         1694028
passenger_count          1694028
trip_distance            1694028
RatecodeID               1694028
PULocationID             1694028
DOLocationID             1694028
payment_type             1694028
fare_amount              1694028
extra                    1694028
mta_tax                  1694028
tip_amount               1694028
tolls_amount             1694028
improvement_surcharge    1694028
total_amount             1694028
congestion_surcharge     1694028
airport_fee              1694028
trip_time                1694028
fare_per_miles           1694028
geometry                 1694028
dtype: int64

Now we visualise total fare Yellow earns, total tips drivers get and total trips in every location ID

In [14]:
# Total Amount
proportions = yellow_df[['PULocationID', 'total_amount']] \
                .groupby('PULocationID') \
                .agg(
                    {
                        'total_amount': 'sum', # sum over total amount earned
                        'PULocationID': 'count' # count number of instances from sample
                    }
                ) \
                .rename({'PULocationID': 'total_trips'}, axis=1)

print(proportions.head())

proportions['proportion_of_total_amount'] = (
    # total amount earned from the Location ID * number of trips in Location / total number of trips
    proportions['total_amount'] * (proportions['total_trips'] / proportions['total_trips'].sum())
)
proportions = proportions.reset_index().sort_values('proportion_of_total_amount', ascending=False)

print(proportions.head())


m = folium.Map(location=[40.73, -73.74], tiles="Stamen Terrain", zoom_start=10)

# refer to the folium documentations on more information on how to plot aggregated data.
c = folium.Choropleth(
    geo_data=geoJSON, # geoJSON 
    name='choropleth', # name of plot
    data=proportions.reset_index(), # data source
    columns=['PULocationID', 'proportion_of_total_amount'], # the total_columns required
    key_on='properties.LocationID', # this is from the geoJSON's properties
    fill_total_amountor='YlOrRd', # color scheme
    nan_fill_total_amountor='black',
    legend_name='Proportion of total earnings USD$'
)

c.add_to(m)


gdf.loc[gdf['Zone'].str.contains('Airport')]
# (y, x) since we want (lat, long)
gdf['centroid'] = gdf['geometry'].apply(lambda x: (x.centroid.y, x.centroid.x))
gdf[['Zone', 'LocationID', 'centroid']].head()
for zone_name, coord in gdf.loc[gdf['Zone'].str.contains('Airport'), ['Zone', 'centroid']].values:
    m.add_child(
        folium.Marker(location=coord, popup=zone_name)
    )
m.save('../plots/yellow/map/yellow_total_earnings.html')

              total_amount  total_trips
PULocationID                           
1                  2306.29           27
2                    88.80            1
3                   119.90            3
4                 40146.13         1686
6                    25.10            1
     PULocationID  total_amount  total_trips  proportion_of_total_amount
118           132    7884583.76        95956               446611.932787
124           138    4380101.64        64546               166891.007974
146           161    2112361.92        82189               102485.268155
217           237    1693947.47        80959                80955.151405
216           236    1508615.79        70558                62835.391688


In [15]:
# Total tips
proportions = yellow_df[['PULocationID', 'tip_amount']] \
                .groupby('PULocationID') \
                .agg(
                    {
                        'tip_amount': 'sum', # sum over total tips earned
                        'PULocationID': 'count' # count number of instances from sample
                    }
                ) \
                .rename({'PULocationID': 'total_trips'}, axis=1)

print(proportions.head())

proportions['proportion_of_tip_amount'] = (
    # total tips earned from the Location ID * number of trips in Location / total number of trips
    proportions['tip_amount'] * (proportions['total_trips'] / proportions['total_trips'].sum())
)
proportions = proportions.reset_index().sort_values('proportion_of_tip_amount', ascending=False)

print(proportions.head())


m = folium.Map(location=[40.73, -73.74], tiles="Stamen Terrain", zoom_start=10)

# refer to the folium documentations on more information on how to plot aggregated data.
c = folium.Choropleth(
    geo_data=geoJSON, # geoJSON 
    name='choropleth', # name of plot
    data=proportions.reset_index(), # data source
    columns=['PULocationID', 'proportion_of_tip_amount'], # the columns required
    key_on='properties.LocationID', # this is from the geoJSON's properties
    fill_total_amountor='YlOrRd', # color scheme
    nan_fill_total_amountor='black',
    legend_name='Proportion of tip earnings USD$'
)

c.add_to(m)


gdf.loc[gdf['Zone'].str.contains('Airport')]
# (y, x) since we want (lat, long)
gdf['centroid'] = gdf['geometry'].apply(lambda x: (x.centroid.y, x.centroid.x))
gdf[['Zone', 'LocationID', 'centroid']].head()
for zone_name, coord in gdf.loc[gdf['Zone'].str.contains('Airport'), ['Zone', 'centroid']].values:
    m.add_child(
        folium.Marker(location=coord, popup=zone_name)
    )
m.save('../plots/yellow/map/yellow_tip_earnings.html')

              tip_amount  total_trips
PULocationID                         
1                 157.09           27
2                  14.80            1
3                   5.00            3
4                5050.37         1686
6                   0.00            1
     PULocationID  tip_amount  total_trips  proportion_of_tip_amount
118           132   859413.33        95956              48680.343828
124           138   569328.91        64546              21692.618909
146           161   272629.22        82189              13227.126684
217           237   220824.29        80959              10553.375561
216           236   198411.79        70558               8264.054124


In [16]:
# Total Trips
m = folium.Map(location=[40.73, -73.74], tiles="Stamen Terrain", zoom_start=10)

# refer to the folium documentations on more information on how to plot aggregated data.
c = folium.Choropleth(
    geo_data=geoJSON, # geoJSON 
    name='choropleth', # name of plot
    data=proportions.reset_index(), # data source
    columns=['PULocationID', 'total_trips'], # the columns required
    key_on='properties.LocationID', # this is from the geoJSON's properties
    fill_total_amountor='YlOrRd', # color scheme
    nan_fill_total_amountor='black',
    legend_name='Proportion of trips'
)

c.add_to(m)


gdf.loc[gdf['Zone'].str.contains('Airport')]
# (y, x) since we want (lat, long)
gdf['centroid'] = gdf['geometry'].apply(lambda x: (x.centroid.y, x.centroid.x))
gdf[['Zone', 'LocationID', 'centroid']].head()
for zone_name, coord in gdf.loc[gdf['Zone'].str.contains('Airport'), ['Zone', 'centroid']].values:
    m.add_child(
        folium.Marker(location=coord, popup=zone_name)
    )
m.save('../plots/yellow/map/yellow_total_trips.html')

In [19]:
fare_per_miles = yellow_df["total_amount"].sum() / yellow_df["trip_distance"].sum()
fare_per_trips = yellow_df["total_amount"].sum() / proportions["total_trips"].sum()
tips_per_trips = yellow_df["tip_amount"].sum() / proportions["total_trips"].sum()
tips_per_miles = yellow_df["tip_amount"].sum() / yellow_df["trip_distance"].sum()
miles_per_trips = yellow_df["trip_distance"].sum() / proportions["total_trips"].sum()

print(f"yellow fare/miles: {fare_per_miles}")
print(f"yellow fare/trips: {fare_per_trips}")
print(f"yellow tips/trips: {tips_per_trips}")
print(f"yellow tips/miles: {tips_per_miles}")
print(f"yellow miles/trips: {miles_per_trips}")

yellow fare/miles: 8.226390600904448
yellow fare/trips: 29.46768865685809
yellow tips/trips: 3.677158022181451
yellow tips/miles: 1.0265392221277516
yellow miles/trips: 3.5820920846644793
